In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


anaconda3     core.6761  cpp-test   gensim-data   opencv_contrib  Templates
aws	      core.6774  Desktop    installation  Pictures	  thc-hydra
awscliv2.zip  core.6824  Documents  Music	  Public	  Videos
core.6709     core.6988  Downloads  opencv	  snap		  workspace


In [2]:
from google.colab import drive
import time, os
import sys

drive.mount("/content/drive")

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = "CV_assignments/Final"
GOOGLE_DRIVE_PATH = os.path.join("drive", "My Drive", GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

sys.path.append(GOOGLE_DRIVE_PATH)

ModuleNotFoundError: ignored

In [ ]:
%%capture

!pip install timm
!pip install einops

In [ ]:
!pip install kymatio

In [ ]:
from timm.data import create_transform
from torchvision import datasets, transforms
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import torch

def build_transform(is_train):
    resize_im = 224 > 32
    if is_train:
        # this should always dispatch to transforms_imagenet_train
        transform = create_transform(
            input_size=224,
            is_training=True,
            color_jitter=0.4,
            auto_augment='rand-m9-mstd0.5-inc1',
            interpolation='bicubic',
            re_prob=0.25,
            re_mode='pixel',
            re_count=1,
        )
        if not resize_im:
            # replace RandomResizedCropAndInterpolation with
            # RandomCrop
            transform.transforms[0] = transforms.RandomCrop(
                224, padding=4)
        return transform

    t = []
    if resize_im:
        size = int((256 / 224) * 224)
        t.append(
            # to maintain same ratio w.r.t. 224 images
            transforms.Resize(size, interpolation=3),
        )
        t.append(transforms.CenterCrop(224))

    t.append(transforms.ToTensor())
    t.append(transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD))
    return transforms.Compose(t)

In [ ]:
from timm.models import create_model
from torchvision.datasets.folder import ImageFolder


transform = build_transform(True)
dataset_train = ImageFolder("/content/drive/MyDrive/ImageNet100/train", transform=transform)

In [ ]:
total_samples = len(dataset_train)

# Calculate the number of samples you want to take (10% in this case)
percent = 0.005
subset_size = int(total_samples * percent)

# Create a SubsetRandomSampler to select the subset of data
subset_sampler = torch.utils.data.SubsetRandomSampler(range(subset_size))

# Create a new data loader for the subset
subset_data_loader = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=32,
    sampler=subset_sampler,
    pin_memory=True,
    drop_last=True,
)

len(subset_data_loader)

19

In [ ]:
data_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=32,
        # num_workers=0,
        pin_memory=True,
        drop_last=True,
    )

In [ ]:
from timm.models import create_model
import torch

teacher_model = create_model(
    'regnety_160',
    pretrained=False,
    num_classes=10,
    global_pool='avg',
)

checkpoint = torch.hub.load_state_dict_from_url(
        'https://dl.fbaipublicfiles.com/deit/regnety_160-a5fe301d.pth', map_location='cpu', check_hash=True)
teacher_model.load_state_dict(checkpoint['model'])

Downloading: "https://dl.fbaipublicfiles.com/deit/regnety_160-a5fe301d.pth" to /root/.cache/torch/hub/checkpoints/regnety_160-a5fe301d.pth
100%|██████████| 319M/319M [00:09<00:00, 36.0MB/s]


RuntimeError: ignored

In [ ]:
from models import *
from timm.models import create_model
model = create_model(
       "scatformer_s0",
       num_classes=1000,
       distillation=("hard" != 'none'),
       pretrained=None,
       fuse=False,
   )

Layer: 0
FNN	 block_idx 0
FNN	 block_idx 1
downsamples: 56
ELSE
Layer: 1
FNN	 block_idx 0
FNN	 block_idx 1
downsamples: 28
ELSE
Layer: 2
FNN	 block_idx 0
FNN	 block_idx 1
FNN	 block_idx 2
FNN	 block_idx 3
AttnFNN	 block_idx 4
AttnFNN	 block_idx 5
downsamples: 14
ASUB
Layer: 3
FNN	 block_idx 0
FNN	 block_idx 1
AttnFNN	 block_idx 2
AttnFNN	 block_idx 3


In [ ]:
model(torch.randn(2, 3, 224, 224))

X shape: torch.Size([2, 3, 224, 224])
X shape: torch.Size([2, 32, 56, 56])
X shape: torch.Size([2, 48, 28, 28])
X shape: torch.Size([2, 96, 14, 14])
X shape: torch.Size([2, 176, 7, 7])


In [ ]:
!sh /content/drive/MyDrive/CV_assignments/Final/dist_train.sh scatformer_s0 1

Not using distributed mode
Namespace(batch_size=64, epochs=50, model='scatformer_s0', input_size=224, model_ema=True, model_ema_decay=0.99996, model_ema_force_cpu=False, opt='adamw', opt_eps=1e-08, opt_betas=None, clip_grad=0.01, clip_mode='agc', momentum=0.9, weight_decay=0.025, sched='cosine', lr=0.001, lr_noise=None, lr_noise_pct=0.67, lr_noise_std=1.0, warmup_lr=1e-05, min_lr=1e-05, decay_epochs=30, warmup_epochs=5, cooldown_epochs=10, patience_epochs=10, decay_rate=0.1, color_jitter=0.4, aa='rand-m9-mstd0.5-inc1', smoothing=0.1, train_interpolation='bicubic', repeated_aug=True, reprob=0.25, remode='pixel', recount=1, resplit=False, mixup=0.8, cutmix=1.0, cutmix_minmax=None, mixup_prob=1.0, mixup_switch_prob=0.5, mixup_mode='batch', teacher_model='regnety_160', teacher_path='https://dl.fbaipublicfiles.com/deit/regnety_160-a5fe301d.pth', distillation_type='none', distillation_alpha=0.5, distillation_tau=1.0, finetune='', data_path='drive/MyDrive/CIFAR10', data_set='CIFAR10', inat_ca